## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset

# Dataset already downloaded
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'All chars:<start>{ ''.join(chars) }<end>' )
print(f'vocab_size: {vocab_size}')

All chars:<start>
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz<end>
vocab_size: 65


In [6]:
# create a mapping from characters to integers
#-------------------------------------------------------------------------
str_to_idx = { # string to index 
                char:idx 
                for idx,char in enumerate(chars) 
             } 

idx_to_str = { # index to string
                idx:char 
                for idx,char in enumerate(chars) 
             } 
#-------------------------------------------------------------------------
encode = lambda str: [str_to_idx[c]  # encoder: take a string, output a list of integers
                      for c in str] 

decode = lambda int_list: ''.join( # decoder: take a list of integers, output a string
                                [ idx_to_str[i] 
                                  for i in int_list ]
                              )
#-------------------------------------------------------------------------

str_test = "hii there"
str_test_encoded = encode(str_test)
str_test_decoded = decode(str_test_encoded)
print(str_test_encoded)
print(str_test_decoded)

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data_selected = torch.tensor(encode(text), dtype=torch.long)
print(data_selected.shape, data_selected.dtype)
print(data_selected[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Let's now split up the data into train and validation sets
n = int( 0.9 * len(data_selected) ) # first 90% will be train, rest val
train_data = data_selected[:n] # from 0 to n-1
validation_data = data_selected[n:] # from n to end

In [9]:
block_size = 8
train_data[:block_size+1] # just testing

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size] # block_size = 8
y = train_data[1:block_size+1]

print(f'original: {train_data[:block_size+1]}')
print(f'       x: {x}')
print(f'       y:     {y}\n')

#-------------------------------------------------------------------------
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when input is {context} the target: {target}")
#-------------------------------------------------------------------------

original: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
       x: tensor([18, 47, 56, 57, 58,  1, 15, 47])
       y:     tensor([47, 56, 57, 58,  1, 15, 47, 58])

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

#-------------------------------------------------------------------------
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data_selected = train_data if split == 'train' else validation_data

    #----------
    # ix = torch.randint(len(data_selected) - block_size, (batch_size,))

    # this is cut short from the len minus the block size, if we select the last possible index, we need to be able
    #   to select block_size elements after it
    high_bound = len(data_selected) - block_size 
    size_selection = (batch_size,)

    # note the high is exclusive
    rand_int_tensor = torch.randint(high=high_bound, size=size_selection) # produces a random integer tensor, where any value is in between 0 and high_bound
    
    # print(f'\nlen(data_selected): {len(data_selected)}, block_size: {block_size}, high_bound: {high_bound}')
    # print(f"size_selection: {size_selection}, ix:{rand_int_tensor}\n\n") # high_bound: 1003846, size_selection: (4,), ix:tensor([ 76049, 234249, 934904, 560986])
    #----------
    
    r'''
    there an concern if this is under list bounds, so let's test the limits.
    consider len(data_selected) = 100, block_size = 8
    then high_bound = 100 - 8 = 92, but since the high is exclusive, the last possible index is 91
    For the First list:
      if 'i' was 91, we would have: start idx = 91 (inclusive), end idx = 91 + 8 = 99 (exclusive)
      therefore we would get: 91, 92, 93, 94, 95, 96, 97, 98

    For the second list:
      if 'i' was 91, then we would have: start idx = i + 1 = 92 (inclusive), end idx = i + 1 + 8 = 91 + 1 + 8 = 100 (exclusive)
      therefore we would get: 92, 93, 94, 95, 96, 97, 98, 99
    '''
    # select a segment starting at index i (from random sampling indexes) with length of block_size
    x = torch.stack( [ data_selected[ i : i+block_size ] 
                      for i in rand_int_tensor 
                      ] 
                    )
    # select a segment virtually identical to x, but shifted by one position to the right
    y = torch.stack( [ data_selected[ i+1 : i+1+block_size ] 
                      for i in rand_int_tensor 
                      ] 
                    )
    return x, y
#-------------------------------------------------------------------------

xb, yb = get_batch('train')
print(f'inputs shape:{xb.shape}')
print(f'inputs:\n{xb}')
print(f'\ntargets shape:{ yb.shape}')
print(f'targets:\n{yb}')

print('----')

#-------------------------------------------------------------------------
for b in range(batch_size): # batch dimension
    for i in range(block_size): # time dimension
        context = xb[b, :i+1] # get all elements from row b, from column 0 to i+1 (non-inclusive) - also remember this is shifted by one behind
        target = yb[b,i] # get element at row b, column i - this is shifted by one ahead
        print(f"when input is {context.tolist()} the target: {target}")
#-------------------------------------------------------------------------

inputs shape:torch.Size([4, 8])
inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets shape:torch.Size([4, 8])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the targ

In [12]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

#-------------------------------------------------------------------------
class BigramLanguageModel(nn.Module):
    #-------------------------------------------------------------------------
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(num_embeddings = vocab_size, embedding_dim = vocab_size)
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    def forward(self, idx, targets=None):

        #note: Logit -> The raw predictions which come out of the last layer of the neural network

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) - typically batch size, block size/ time steps, vocab size/channels
        
        # #-------
        # print(f'idx shape: {idx.shape}\nidx:\n{idx}')
        # if targets is not None:
        #     print(f'targets shape: {targets.shape}\ntargets:\n{targets}')
        # print('---------------')
        # print(f'logits shape: {logits.shape}\nlogits: (they are the idx that went through token_embedding_table)\n{logits}')
        # #-------

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch size (4), time steps (8), channels (65)
            logits = logits.view(B*T, C) # we reorganize and keep the channels/ vocab size as the last and isolated dimension
            targets = targets.view(B*T) # target was not modified and has shape (4, 8)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            # we provide the idx, which is transformed via the token_embedding_table (nn.Embedding) 
            logits, _loss = self(idx = idx) # get the predictions - the problem here is that we are transforming the embedding multiple times

            # focus only on the last time step, select all rows, but only the last column, and all channels
            logits = logits[:, -1, :] # from (B,T,C) becomes (B, C)
            
            # apply softmax to get probabilities - dim=-1 means we apply softmax to the last dimension
            probs = F.softmax(logits, dim=-1) # (B, C) -> (1, 65) - we could skip this step
            
            # draw samples (1) from the distribution - the return is a index from the tensor
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) - we could pick a random number between 0 and vocab_size
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    #-------------------------------------------------------------------------
#-------------------------------------------------------------------------
m = BigramLanguageModel(vocab_size = vocab_size) # vocab_size = 65
logits, loss = m(idx = xb, targets = yb)
print(f'logits shape (note 4 * 8 = 32): {logits.shape}\nlogits:\n{logits}')
print(f'loss:{loss}')

temp_torch_zeros = torch.zeros(size = (1, 1), dtype=torch.long) # one row one column: [[0]]
temp_m_generate = m.generate(
    idx             = temp_torch_zeros, 
    max_new_tokens  = 100
)
temp_m_generate_list = temp_m_generate[0].tolist()
temp_decode = decode( temp_m_generate_list )


# print(f'temp_torch_zeros: {temp_torch_zeros}')
# print(f'temp_m_generate:\n{temp_m_generate}')
# print(f'temp_m_generate_list:{temp_m_generate_list}')

print( temp_decode )

logits shape (note 4 * 8 = 32): torch.Size([32, 65])
logits:
tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)
loss:4.878634929656982

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
#-------------------------------------------------------------------------
# loop_steps = 100_000
loop_steps = 100
for steps in range(loop_steps): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train') # as usual 'yb' is shifted by one position to the right

    # evaluate the loss (and do the embeddings)
    logits, _loss = m(idx = xb, targets = yb)
    # print(f'    _loss: {_loss.item()}')

    # set gradients to None - this will have a lower memory footprint and slightly improve performance
    optimizer.zero_grad(set_to_none=True) 

    _loss.backward()
    optimizer.step()
#-------------------------------------------------------------------------

print(f'With Optimizer\nFinal Loss: {_loss.item()}') 

With Optimizer
Final Loss: 2.404492139816284


In [16]:
temp_torch_zeros = torch.zeros((1, 1), dtype=torch.long)
temp_m_generate = m.generate(
    idx             = temp_torch_zeros, 
    max_new_tokens  = 500
)

temp_m_generate_list = temp_m_generate[0].tolist()

print(decode(temp_m_generate_list))


Bere ak hongacl GELusey wssisean thi's.
When, w t med yfthon oklichay ol, I INI' dede blais cicen m pto y sooninde, thind w:
Qurrsad muee?
BYoverd le
ANVINThaich an, Se dife bl, t hino l. ghat aven's awis.
Wh malloo be ndema pusthr
WADINEDom osw sale t ve,
IR:
Whwarr cha mandu d tours nofok s t f is fo wighathalt
SSis.
WAsper;
ce g CHA:
Whamasomasthens Joredasthereath istor itoutourtoumervesofechespeaksts
I the'tou n:
ARThur o ay
A:
HELARindend s, foiowlonifene m atho pl:
ler:
Y ar.
Thicheangoon


## The mathematical trick in self-attention

In [17]:
#-------------------------------------------------------------------------
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
temp_ones = torch.ones(3, 3)
a = torch.tril(temp_ones) # return a lower triangular part of the matrix

# computes the sum of elements along a specified dimension of a tensor
sum_a = torch.sum(input = a, dim = 1, keepdim=True) 

a = a / sum_a
b = torch.randint( low = 0, high = 10, size = (3,2) ).float() # size (3,2) - 3 rows, 2 columns
c = a @ b # matrix multiplication introduced in Python 3.5
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
#-------------------------------------------------------------------------

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for i in range(T):
        xprev = x[b,:i+1] # (t,C)
        xbow[b,i] = torch.mean(xprev, 0)


In [20]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [21]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [22]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

In [ ]:
q.var()

In [ ]:
wei.var()

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
str_to_idx = { ch:i for i,ch in enumerate(chars) }
idx_to_str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [str_to_idx[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([idx_to_str[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data_selected = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data_selected)) # first 90% will be train, rest val
train_data = data_selected[:n]
validation_data = data_selected[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, _loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    _loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4195, val loss 2.4336
step 400: train loss 2.3508, val loss 2.3572
step 500: train loss 2.2960, val loss 2.3125
step 600: train loss 2.2409, val loss 2.2497
step 700: train loss 2.2053, val loss 2.2186
step 800: train loss 2.1640, val loss 2.1873
step 900: train loss 2.1245, val loss 2.1503
step 1000: train loss 2.1027, val loss 2.1293
step 1100: train loss 2.0701, val loss 2.1196
step 1200: train loss 2.0383, val loss 2.0798
step 1300: train loss 2.0252, val loss 2.0638
step 1400: train loss 1.9924, val loss 2.0368
step 1500: train loss 1.9704, val loss 2.0309
step 1600: train loss 1.9615, val loss 2.0474
step 1700: train loss 1.9408, val loss 2.0126
step 1800: train loss 1.9083, val loss 1.9944
step 1900: train loss 1.9078, val loss 1.9873
step 2000: train loss 1.8867, val loss 1.9963
step 2100: train loss 1.

In [20]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


BUCKINGHAM:
Then, his lost to betchsed ingrence I make I me, slate ine,
A carm tere bote recting do.
Cist, onames I't have becond prinet,
Contman's bsorne would swilistip
Lords thee heir beast miont my his conforn many
his med hath corner to more you
And smence! with the man unter some sonstiful.

CAPSON:
Here is in no approuble ill.

DUKE OW MARCHIUS:
Goods may my proyate 's my but
Dear our somet.

JOY BOLINGBROKE:
I markly be aste-timain: that deed,
Go rohpred will know our my streque,
So self you lield lit. You hast full-dight,
Wheret is the gets are I am you.

BOLNGBROMY:
As I he wind, I wast grove wove to the portrey.

Shall Reamors.

Merry.

PARGARET:
Every me padty me were is tonce:
A thounful themm set it! thou womay shrown kingd with arm fortunce
To clitten saugh rett
And maker what his father alling
Cry a knowly the brace is thunged Donguere: to me,
What's. Yet you ham, with stome, that her to brut as grubt critus you'll'st
we by to Right outt, resperect, to I reford
What's 